In [1]:
import re
import pandas as pd
import numpy as np
import string
from datetime import datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window

#chdrpf_path = '/group/axa_malaysia/data/adm_chdrpf'

def policy_psea(chdrpf_path, output_folder='data/sas_401/'):
    
    """
    def strip_punctuation(s):
        s = str(s)
        s= ''.join(c for c in s if c not in string.punctuation)
        s.replace(" ",'')
        return(s)
    strip_punctuationudf = udf(strip_punctuation)
    
    # Construct regular expression to match criteria of valid replacement, criteria being
    # start with P/Q/S and has 7 digits.
    def check(x):
         bool(re.search(('^[PQS]\d{7}$'),str(x)))            
    checkudf = udf(check,BooleanType())
    
    def thirdcheck(x):
        bool(x.startswith(tuple(['P','Q','S'])))
    thirdcheckudf = F.udf(thirdcheck,BooleanType())

    def fourthcheck(x):
        bool(re.search('[a-z]',str(x)[1:]) is not None)
    fourthcheckudf = F.udf(fourthcheck,BooleanType())
    """
    
    def format_date(strdate):
        try: 
            return datetime.strptime(str(strdate),'%Y%m%d').strftime('%Y-%m-%d')
        except: 
            return '2999-12-31'
    _format_date = udf(format_date,StringType())
    
    def _replace_null_dates(column):
        return when(column.isNull(), to_date(lit('2999-12-31'))).otherwise(column)
        
    def _get_replacements(policyNo, commencementDate, replacementNo):
        ziplist = zip(policyNo, commencementDate, replacementNo)
        sorted_policy = sorted(ziplist, key=lambda x: (x[1],x[0]), reverse=True )
        chain_list = []
        policy_chainnum = {}

        for pol, com, rep in sorted_policy:
            if pol in policy_chainnum:
                chain_list[policy_chainnum[pol]].append( (pol,com) )
                if rep:
                    policy_chainnum[rep] = policy_chainnum[pol]
            else:
                policy_chainnum[pol] = len(chain_list)
                chain_list.append([(pol,com)])
                if rep:
                    policy_chainnum[rep] = policy_chainnum[pol]

        final_output = []
        for chain in chain_list:
            for idx, val in enumerate(chain):
                final_output.append({'chdrnum': val[0],
                                    'orig_pol':chain[-1][0],
                                     'latest_replacement':chain[0][0],
                                     'd_first_com':chain[-1][1],
                                     'update_count': len(chain)-idx-1 })

        return final_output
    _get_replacements_udf = udf(_get_replacements, ArrayType(StructType([
                    StructField("d_first_com", DateType()),
                    StructField("latest_replacement", StringType()),
                    StructField("orig_pol", StringType()),
                    StructField("update_count", IntegerType()),
                    StructField("chdrnum", StringType())])))

    # Keep only
    # 1) FG, hence exclude G400, set servunit = FG which is Fire and General;
    # 2) In Force or Cancelled - statcode = IF and CA records from CHDRPF
    chdrpf = spark.read.parquet(chdrpf_path).select('SERVUNIT','CHDRNUM','CURRFROM','CURRTO',
                          'TRANNO','CHDRSTCDC','CNTTYPE','COWNNUM',
                          'OCCDATE','CNTBRANCH','RNLTYPE','NOFRISKS',
                          'DTECAN','REPTYPE','ZREPOLNO','REPNUM',
                          'STATCODE','AGNTNUM','CCDATE','CRDATE','ZENDNO',
                          'ZRENNO','STATTRAN','CAMPAIGN').filter((col('SERVUNIT')=='FG') & (col('STATCODE').isin ('IF','CA')))

    # Sort in this order to ensure most recent is kept - zrenno is included since tranno is not necessarily chronological
    chdrpf = chdrpf.orderBy(['CHDRNUM','ZRENNO','CURRTO','TRANNO'],ascending=[1,0,0,0])
    # Remove duplicates by taking keeping the most recent information for each policy for each POI
    chdrpf = chdrpf.dropDuplicates(['CHDRNUM','ZRENNO','CURRTO','TRANNO'])

    # Clean-up fields
    polhistory_psea = chdrpf.withColumn("ID",lit("PSEA")) # Important when G400 is added

    # 1) Effective dates on policy version - thus could be for each policy version throughout the year,
    #    hence there could be many changes per policy
    polhistory_psea = polhistory_psea.withColumn("D_FROM", to_date(_format_date(col('CURRFROM'))))
    # 2) Same as above, just the date until the change is effective
    polhistory_psea = polhistory_psea.withColumn("D_TO", to_date(_format_date(col('CURRTO'))))
    # 3) Original commencement date for current policy version it does not look at the original dates for replaced contracts,
    #    thus there is only one for each policy version
    polhistory_psea = polhistory_psea.withColumn("D_ORICOM", to_date(_format_date(col('OCCDATE'))))
    # 4) Commencement date, date on which policy is renewed (or new business) this is the start of the POI
    polhistory_psea = polhistory_psea.withColumn("D_COM", to_date(_format_date(col('CCDATE'))))
    # 5) Renewal date, if not renewed then expired
    polhistory_psea = polhistory_psea.withColumn("D_EXP", to_date(_format_date(col('CRDATE'))))
    # 6) Date of cancellation
    polhistory_psea = polhistory_psea.withColumn("D_cancel", when(col('STATCODE')=='CA', to_date(_format_date(col('DTECAN')))))\
    .withColumn("D_cancel", _replace_null_dates(col('D_cancel')))

    # Rename fields
    polhistory_psea = polhistory_psea.withColumnRenamed("AGNTNUM","AGENTID").withColumnRenamed("COWNNUM","OWNERID")

    # Renewability
    polhistory_psea = polhistory_psea.withColumn("RENEWABLE",when(col('RNLTYPE').isin(['M2','02']),lit(0)).otherwise(lit(1)))

    # Clean-up replacements
    # Remove trailing blanks and keep the number only if:
    # 1. REPTYPE is RF or RX - replacements occur only when REPTYPE is RF or RX
    # 2. REPNUM is not:
    #    2(a): CHDRNUM, and
    #    2(b): Length of REPNUM is exactly 8 with
    #       2(b)(i)  : first character is a letter
    #       2(b)(ii) : all the rest are digits
    #       2(b)(iii): Careful with GTOM (Structure might be different)
    # Same conditions apply on ZREPOLNO

    # Replace blanks
    polhistory_psea = polhistory_psea.withColumn('CHDRNUM', translate('CHDRNUM', string.punctuation+' ', ""))
    polhistory_psea = polhistory_psea.withColumn('REPNUM', translate('REPNUM', string.punctuation+' ', ""))
    polhistory_psea = polhistory_psea.withColumn('ZREPOLNO', translate('ZREPOLNO', string.punctuation+' ', ""))

    # Clear if condition 1 above is not met - if it is anything other than
    polhistory_psea = polhistory_psea.withColumn('REPNUM',when(~col('REPTYPE').isin(['RF','RX','CH']),"").otherwise(col('REPNUM')))
    polhistory_psea = polhistory_psea.withColumn('zrepolno',when(~col('REPTYPE').isin(['RF','RX','CH']),"").otherwise(col('ZREPOLNO')))

    # chdrpf = chdrpf.withColumn('REPNUM', when(col('REPNUM') != F.lit(None) &
    #                                          ((length(col('REPNUM'))!=8) |
    #                                          (thirdcheckudf(col('REPNUM'))) |
    #                                          (fourthcheckudf(col('REPNUM')))),"").otherwise(col('REPNUM')))

    # Fixing repnum based on conditions imposed
    #polhistory_psea = polhistory_psea.withColumn('REPNUM', regexp_extract('REPNUM', '^[PQS]\d{7}$', 0))
    # Fixing zrepolno based on conditions imposed
    #polhistory_psea = polhistory_psea.withColumn('ZREPOLNO', when(col('CHDRSTCDC')=='001', regexp_extract('ZREPOLNO', '^[PQS]\d{7}$', 0))\
    #                                             .otherwise(""))

    polhistory_psea = polhistory_psea.withColumn('REPLNUM',col('REPNUM'))
    polhistory_psea = polhistory_psea.withColumn('COMPNUM',col('ZREPOLNO'))

    # Clear if condition 2(a) above is met
    polhistory_psea = polhistory_psea.withColumn('REPNUM',when(col('REPNUM')==col('CHDRNUM'),"").otherwise(col('REPNUM')))
    polhistory_psea = polhistory_psea.withColumn('ZREPOLNO',when(col('ZREPOLNO')==col('CHDRNUM'),"").otherwise(col('ZREPOLNO')))

    # The replacement number if the replacement number equals the contract number - this cannot be a replacement
    polhistory_psea = polhistory_psea.drop('CURRFROM','CURRTO','RNLTYPE','OCCDATE','CCDATE','CRDATE','DTECAN','ZREPOLNO','REPNUM')
    polhistory_psea.cache()

    # Create Policy Header table
    pol_psea = polhistory_psea.select('CHDRNUM','D_ORICOM','D_CANCEL','CHDRSTCDC','OWNERID','AGENTID','CNTTYPE','ZRENNO','REPLNUM',
                                      'COMPNUM','CNTBRANCH','ID')
    pol_psea = pol_psea.dropDuplicates(['CHDRNUM'])
    pol_psea = pol_psea.orderBy('CHDRNUM')
    pol_psea.cache()

    ## Replacement section
    pol_psea_replacement = pol_psea.select('CHDRNUM','D_ORICOM','OWNERID','CHDRSTCDC','REPLNUM')

    pol_psea_grouped = pol_psea_replacement.groupBy(['OWNERID','CHDRSTCDC']).agg(collect_list('CHDRNUM').alias('CHDRNUM'),
                                                                                 collect_list('D_ORICOM').alias('D_ORICOM'),
                                                                                 collect_list('REPLNUM').alias('REPLNUM'))

    pol_psea_chain = pol_psea_grouped.\
    withColumn('Policy_Chain', _get_replacements_udf(col('CHDRNUM'),col('D_ORICOM'),col('REPLNUM')) ).\
    drop('CHDRNUM','D_ORICOM','REPLNUM')

    pol_psea_exploded = pol_psea_chain.withColumn('Policy_Chain', explode(col('Policy_Chain')) )

    reps2 = pol_psea_exploded.withColumn('d_first_com', col('Policy_Chain')['d_first_com'])\
    .withColumn('latest_replacement', col('Policy_Chain')['latest_replacement'])\
    .withColumn('orig_pol', col('Policy_Chain')['orig_pol'])\
    .withColumn('update_count', col('Policy_Chain')['update_count'])\
    .withColumn('chdrnum', col('Policy_Chain')['chdrnum'])\
    .drop('Policy_Chain')

    # Merge
    cond2 = [pol_psea['CHDRNUM'] == reps2['CHDRNUM']]
    pol_psea = pol_psea.join(reps2, cond2, 'inner').select([pol_psea[xx] for xx in pol_psea.columns] + [reps2['ORIG_POL'],
                                                                                                        reps2['LATEST_REPLACEMENT'],
                                                                                                        reps2['D_FIRST_COM']])
    pol_psea = pol_psea.toDF(*[c.lower() for c in pol_psea.columns])
    pol_psea.write.parquet('{}transv_pol_psea.parquet'.format(output_folder))

    cond3 = [polhistory_psea['CHDRNUM'] == reps2['CHDRNUM']]
    polhistory_psea = polhistory_psea.join(reps2, cond3, 'inner').select([polhistory_psea[xx] for xx in polhistory_psea.columns] + 
                                                                         [reps2['ORIG_POL'],
                                                                          reps2['LATEST_REPLACEMENT'],
                                                                          reps2['D_FIRST_COM'],
                                                                          reps2['UPDATE_COUNT']])
    polhistory_psea = polhistory_psea.toDF(*[c.lower() for c in polhistory_psea.columns])
    polhistory_psea.write.parquet('{}transv_polhistory_psea.parquet'.format(output_folder))

In [ ]:
chdrpf_path = '/group/axa_malaysia/data/adm_chdrpf'
policy_psea(chdrpf_path)

In [3]:
spark.read.parquet('data/sas_401/transv_polhistory_psea.parquet').count()

15105842

In [4]:
spark.read.parquet('data/sas_401/transv_pol_psea.parquet').count()

9363121

In [1]:
!hdfs dfs -get /group/axa_malaysia/raw/OSPREY/02_ACTLIB/01_Main/03_Transversal/polhistory_psea.sas7bdat

In [2]:
import pandas as pd

In [3]:
polhistory_psea_sas = pd.read_sas('polhistory_psea.sas7bdat')

In [5]:
polhistory_psea_sas.to_pickle('polhistory_psea.pickle')

In [1]:
import pickle
import pandas as pd

In [17]:
!ls -la

total 31860308
drwx------   24 cchin platform-users        4096 Aug 25 09:53 .
drwxr-xr-x. 190 root  root                  4096 Aug 23 20:05 ..
-rw-r-----    1 cchin platform-users      234787 Jun 21 11:06 ADM Mapping.xlsm
drwxr-x---   21 cchin platform-users        4096 Jul  4 15:35 anaconda3
-rw-r-----    1 cchin platform-users   523283080 Jun  8 12:19 Anaconda3-4.4.0-Linux-x86_64.sh
-rw-------    1 cchin platform-users        8274 Aug 24 16:15 .bash_history
-rw-------    1 cchin platform-users          18 May 29 18:47 .bash_logout
-rw-------    1 cchin platform-users         176 May 29 18:47 .bash_profile
-rw-------    1 cchin platform-users         124 May 29 18:47 .bashrc
drwxr-x---    2 cchin platform-users        4096 Aug 24 16:15 .beeline
drwxr-x---    3 cchin platform-users        4096 Jun  8 14:26 .cache
-rwxrwxrwx    1 cchin domain users           597 Jul  4 16:49 cchin.conf
-rwxrwxr--    1 cchin root                   394 May 29 18:46 cchin.keytab
-rw-r-----    1 cchin plat

In [2]:
polhistory_psea_sas = pickle.load(open('polhistory_psea.pickle','rb'))

In [55]:
[i.decode('utf-8') for i in polhistory_psea_sas[pd.isnull(polhistory_psea_sas['d_exp'])]['chdrnum']]

['UB017745',
 'UF027805',
 'UJ003602',
 'UM010006',
 'UM126230',
 'UM126231',
 'UM126232',
 'UM126233',
 'UM127438',
 'UO020866',
 'UO159076',
 'UQ028308',
 'UQ036847',
 'UZ011672',
 'UZ012177',
 'UZ027349',
 'UZ033000',
 'UZ034047',
 'UZ034050',
 'UZ034097',
 'UZ490007',
 'UZR07320',
 'UZR07321']

In [40]:
dates_group_sas = polhistory_psea_sas.groupby('d_exp').size()

In [46]:
dates_group_sas.shape

(8936,)

In [31]:
dates_group_sas = polhistory_psea_sas.groupby('d_exp').size()
dates_group_sas = dates_group_sas.reset_index()
dates_group_sas['d_exp'] = dates_group_sas['d_exp'].map(lambda x: x if x <= 60000 else 60000.0)
dates_group_sas['d_exp'] = pd.to_timedelta(dates_group_sas['d_exp'], unit='D') + pd.datetime(1960, 1, 1)
dates_group_sas = dates_group_sas.rename(columns={0:'count'})
#dates_group_sas.to_csv('dates_group_sas_cancel.csv',index=False)

In [32]:
dates_group_sas

,d_exp,count
0,1900-05-03,5
1,1901-07-01,1
2,1973-12-31,1
3,1983-06-30,1
4,1984-05-31,1
5,1984-12-25,1
6,1985-05-05,1
7,1986-07-11,1
8,1986-12-31,1
9,1988-01-26,1


In [30]:
polhistory_psea_spark = spark.read.parquet("/user/cchin/data/sas_401/transv_polhistory_psea.parquet")
polhistory_psea_spark.groupby('d_cancel').count().coalesce(1).write.csv('dates_group_spark_cancel.csv')
!hdfs dfs -get dates_group_spark_cancel.csv

In [37]:
polhistory_psea_spark[['d_exp']].distinct().sort('d_exp').show(1000)

+----------+
|     d_exp|
+----------+
|1900-05-03|
|1901-07-01|
|1973-12-31|
|1983-06-30|
|1984-05-31|
|1984-12-25|
|1985-05-05|
|1986-07-11|
|1986-12-31|
|1988-01-26|
|1988-05-06|
|1988-08-12|
|1988-08-29|
|1989-03-02|
|1990-01-01|
|1990-07-20|
|1990-12-31|
|1991-01-31|
|1991-06-30|
|1991-07-31|
|1991-08-30|
|1991-10-11|
|1992-01-10|
|1992-06-07|
|1992-06-26|
|1992-07-01|
|1992-07-31|
|1992-09-10|
|1992-12-10|
|1993-01-01|
|1993-01-14|
|1993-03-31|
|1993-04-12|
|1993-04-23|
|1993-04-30|
|1993-05-31|
|1993-06-30|
|1993-07-03|
|1993-07-28|
|1993-09-10|
|1993-12-17|
|1993-12-31|
|1994-01-04|
|1994-01-14|
|1994-03-02|
|1994-04-30|
|1994-06-20|
|1994-06-30|
|1994-07-15|
|1994-08-14|
|1994-08-22|
|1994-09-17|
|1994-09-30|
|1994-10-31|
|1994-11-23|
|1994-11-30|
|1994-12-01|
|1994-12-19|
|1994-12-31|
|1995-01-01|
|1995-01-05|
|1995-01-07|
|1995-01-14|
|1995-01-31|
|1995-02-28|
|1995-03-27|
|1995-03-31|
|1995-05-30|
|1995-06-30|
|1995-07-03|
|1995-07-22|
|1995-07-31|
|1995-08-02|
|1995-08-24|

In [15]:
datetime.strptime('19990228', '%Y%m%d').strftime('%Y-%m-%d')

'1999-02-28'

In [1]:
import re
import pandas as pd
import numpy as np
import string
from datetime import datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window
import datetime
import calendar
from dateutil.relativedelta import relativedelta

In [2]:
acc_mth = 7
acc_yr = 2017
transv_pr_gr_psea_path = 'data/sas_402/transv_pr_gr_psea.parquet'
transv_polhistory_psea_path = 'data/sas_401/transv_polhistory_psea.parquet'
transv_p2_path = 'data/sas_405/transv_p2.parquet'
riskpf_path = '/group/axa_malaysia/data/adm_riskpf'

In [84]:
start_date = datetime.date(acc_yr-4,1,1)
end_date = datetime.date(acc_yr,acc_mth,calendar.monthrange(acc_yr, acc_mth)[1])

result = []
today = end_date
current = start_date
while current <= end_date:
    result.append(datetime.date(current.year,current.month,calendar.monthrange(current.year, current.month)[1]))
    current += relativedelta(months=1)

In [85]:
result

[datetime.date(2013, 1, 31),
 datetime.date(2013, 2, 28),
 datetime.date(2013, 3, 31),
 datetime.date(2013, 4, 30),
 datetime.date(2013, 5, 31),
 datetime.date(2013, 6, 30),
 datetime.date(2013, 7, 31),
 datetime.date(2013, 8, 31),
 datetime.date(2013, 9, 30),
 datetime.date(2013, 10, 31),
 datetime.date(2013, 11, 30),
 datetime.date(2013, 12, 31),
 datetime.date(2014, 1, 31),
 datetime.date(2014, 2, 28),
 datetime.date(2014, 3, 31),
 datetime.date(2014, 4, 30),
 datetime.date(2014, 5, 31),
 datetime.date(2014, 6, 30),
 datetime.date(2014, 7, 31),
 datetime.date(2014, 8, 31),
 datetime.date(2014, 9, 30),
 datetime.date(2014, 10, 31),
 datetime.date(2014, 11, 30),
 datetime.date(2014, 12, 31),
 datetime.date(2015, 1, 31),
 datetime.date(2015, 2, 28),
 datetime.date(2015, 3, 31),
 datetime.date(2015, 4, 30),
 datetime.date(2015, 5, 31),
 datetime.date(2015, 6, 30),
 datetime.date(2015, 7, 31),
 datetime.date(2015, 8, 31),
 datetime.date(2015, 9, 30),
 datetime.date(2015, 10, 31),
 dateti

In [3]:
chdrpf_path = '/group/axa_malaysia/data/adm_chdrpf'

In [86]:
p2_pif_sas = pickle.load(open('p2_pif.pickle','rb'))

In [ ]:
p2_pif_spark = spark.read.parquet("/user/cchin/data/sas_405/transv_p2_pif.parquet")